# RDI sensitivity

In [1]:
%pylab inline --no-import-all
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
matplotlib.rcParams['axes.grid'] = False
import sys
import os

folder = '../../../../crispy'
print(folder)
if folder not in sys.path: sys.path.append(folder)

from crispy.params import Params
folder = '../../../crispy'
par = Params(folder)

from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.IFS import polychromeIFS
from crispy.IFS import reduceIFSMap
from crispy.tools.imgtools import scale2imgs,bowtie
from crispy.tools.image import Image
from crispy.tools.rotate import rotateCube,shiftCube
from crispy.tools.postprocessing import mf,recenter_offaxis
from crispy.tools.inputScene import adjust_krist_header
from crispy.tools.detector import averageDetectorReadout
from astropy.io import fits
import astropy.units as u
from crispy.tools.postprocessing import construct_mflib,convolved_mf



Populating the interactive namespace from numpy and matplotlib
../../../../crispy


## Parameters

In [2]:
lamc=660.
BW = 0.18
par.nonoise=True
sampling=2.0
par.lenslet_sampling=1./sampling
folder = '/Users/mrizzo/IFS/OS5_RDI_660_'+str(sampling)
offaxis_psf_filename='/Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits'
OS5_files = '/Users/mrizzo/IFS/OS5/with_lowfc/'
# folder = '/local/data/nicolaus2/mrizzo/OS5_RDI_660_'+str(sampling)
# offaxis_psf_filename='/local/data/nicolaus2/mrizzo/haystacks/for_gsfc/spc_offaxis_psf.fits'
# OS5_files = '/local/data/nicolaus2/mrizzo/haystacks/for_gsfc/with_lowfc/'
IWA=2.7
OWA=8.7
try:
    os.makedirs(folder)
    os.makedirs(folder+'/average')
    os.makedirs(folder+'/detector')  
except OSError:
    pass
par.hdr


SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT                                                                         
COMMENT ************************************************************            
COMMENT ********************** General parameters ******************            
COMMENT ************************************************************            
COMMENT                                                                         
NLENS   =                  108 / # lenslets across array                        
PITCH   =             0.000174 / Lenslet pitch (meters)                         
INTERLAC=                    2 / Interlacing                                    
PHILENS =    26.565051177077

## Process all files from Krist

Only need to do this the first time

In [ ]:
from crispy.tools.postprocessing import processReferenceCubes,processTargetCubes
import astropy.units as u
import glob
filelist = glob.glob(OS5_files+'*.fits')
filelist.sort()
Nref=30
reffiles = filelist[:Nref]
targetfiles =filelist[Nref:]

# ref_outlist = processReferenceCubes(par,xshift=0.0,yshift=0.0,order=3,
#                 outdir_time_series = folder,
#                 ref_input_list=reffiles,
#                 ref_star_T=9377*u.K, ref_star_Vmag=2.37,
#                 lamc=lamc,BW = BW,
#                 tel_pupil_area=3.650265060424805*u.m**2)
# target_outlist = processTargetCubes(par,target_file_list=targetfiles,
#                 outdir_time_series = folder,
#                 target_star_T=5887*u.K, target_star_Vmag=5.03,
#                 lamc=lamc,BW = BW,
#                 tel_pupil_area=3.650265060424805*u.m**2)


## Run RDI

In [ ]:
from crispy.tools.postprocessing import RDI_noise


offsets = [(0.5,0.5),(0.5,-0.5),(0,0)]

results = np.zeros((19,len(offsets)))
for i in range(len(offsets)):
        offset = offsets[i]
        rootname='x%.2f_y%.2f' % (offset[0],offset[1])
        try:
            os.makedirs(folder+'/average/'+rootname)
        except OSError:
            pass

        results[:,i] = RDI_noise(par,offset[0],offset[1],order=3,
                rootname=rootname,
                process_cubes=True,
                process_offaxis_files=True,
                process_detector=True,
                take_averages=True,
                countershift=True,
                normalize_cubes=True,
                ref_only=True,
                IWA=IWA,
                OWA=OWA,
                forced_inttime_ref = 50,                   
                outdir_time_series = folder,
                outdir_detector=folder+'/detector',
                outdir_average=folder+'/average/'+rootname,
                psf_time_series_folder=OS5_files,
                offaxis_psf_filename=offaxis_psf_filename,
                mflib = '',#folder+'/average/mflib.fits.gz''',
                lamc=lamc,BW=BW,
                offaxis_reduced = folder+'/average/offaxis_detector_red_optext.fits')
outkey = fits.HDUList(fits.PrimaryHDU(results.astype(np.float)))
outkey.writeto(folder+'/average/results_rdi'+str(sampling)+'.fits',clobber=True)


crispy - INFO - Read data from HDU 0 of /Users/mrizzo/IFS/OS5/with_lowfc/os5_spc_001.fits
crispy - INFO - X,Y Shift in px in original cubes: 1.12, 3.35
crispy - INFO - Processing file os5_spc_001.fits
crispy - INFO - Shifting input cube
crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Done.
crispy - INFO - Performance: 36 seconds total
crispy - INFO - Writing data to /Users/mrizzo/IFS/OS5_RDI_660_2.0/os5_spc_001_refstar_IFS.fits
crispy - INFO - Processing file os5_spc_002.fits
crispy - INFO - Shifting input cube
crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Done.
crispy - INFO - Performance: 37 seconds total
crispy - INFO - Writing data to /Users/mrizzo/IFS/OS5_RDI_660_2.0/os5_spc

In [ ]:
plt.figure(figsize=(12,12))
results=Image(folder+'/average/results_rdi'+str(sampling)+'.fits').data
plt.imshow(results,cmap='Greys_r')
plt.colorbar()

In [ ]:
plt.figure(figsize=(15,12))
trim=31
slicenum=2
fontsize=11
plt.subplot(331)
img1 = Image(folder+'/average/x0.00_y0.00/ref_average_detector_countershifted_x0.00_y0.00_red_optext.fits').data
plt.imshow(img1[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Reference star centered',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(332)
img2 = Image(folder+'/average/x0.50_y0.50/ref_average_detector_countershifted_x0.50_y0.50_red_optext.fits').data
plt.imshow(img2[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Reference star shifted by (0.5,0.5) lenslets',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(333)
diff = img2-img1
plt.imshow(diff[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Difference (after countershift)',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(334)
img1 = Image(folder+'/average/x0.00_y0.00/convolved_without_RDI_reference.fits').data
plt.imshow(img1[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Ref star convolved with matched filter',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(335)
img2 = Image(folder+'/average/x0.50_y0.50/convolved_without_RDI_reference.fits').data
plt.imshow(img2[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Countershifted ref convolved with matched filter',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(336)
diff = img2-img1
plt.imshow(diff[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Difference',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(337)
img1 = Image(folder+'/average/x0.00_y0.00/convolved_with_RDI.fits').data
plt.imshow(img1[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Science image residuals post-RDI',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(338)
img2 = Image(folder+'/average/x0.50_y0.50/convolved_with_RDI.fits').data
plt.imshow(img2[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Residuals post-RDI, countershifted reference frame',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(339)
diff = img2-img1
plt.imshow(diff[slicenum,trim:-trim,trim:-trim], cmap='viridis')
plt.title('Difference',fontsize=fontsize)
plt.colorbar(fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(folder+'/average/test.png')

In [ ]:
bowtie = fits.getdata(folder+'/average/x0.00_y0.00/bowtie.fits')
matched_filter = fits.getdata(folder+'/average/x0.00_y0.00/mflib.fits.gz')
data = fits.getdata(folder+'/average/x0.50_y0.50/ref_average_detector_countershifted_x0.50_y0.50_red_optext.fits')
print matched_filter.shape
plt.figure(figsize=(20,12))
plt.subplot(121)
plt.imshow(np.nansum(matched_filter,axis=0)[1]*bowtie[30:-30,30:-30], cmap='viridis')
plt.colorbar(fraction=0.046, pad=0.04)
plt.subplot(122)
plt.imshow(bowtie[30:-30,30:-30]*data[2,30:-30,30:-30], cmap='viridis')
plt.colorbar(fraction=0.046, pad=0.04)


In [ ]:
# img1 = Image(folder+'/average/x0.50_y0.50/ref_average_detector_x0.50_y0.50_red_optext.fits').data
# import scipy
# c = np.cos(par.philens)
# s = np.sin(par.philens)
# xshift=10
# yshift=10
# img1[np.isnan(img1)]=0.0
# print yshift*c+xshift*s,xshift*c-yshift*s
# img1 = scipy.ndimage.interpolation.shift(img1,
#                                     [0.0,xshift*c-yshift*s,yshift*c+xshift*s],order=3)
# Image(data=img1).write(folder+'/average/x0.50_y0.50/'+"test.fits")
